# TDDFT calculations of HOMO-LUMO and singlet-triplet energy gaps of TADF with solvation and dispersion
*June 2024*

## Introduction 

  Les diodes électroluminescentes organiques ou OLED sont devenues de plus en plus populaires ces dernières années en tant que base pour la fabrication d'écrans de télévision minces et flexibles et de téléphones portables qui émettent de la lumière lors de l'application d'un courant électrique. 
  
  Les émetteurs TADF pourraient potentiellement produire des OLED qui fonctionnent avec une efficacité quantique interne (IQE) de 100%, c'est-à-dire la fraction des porteurs de charge dans un circuit ou un système qui émettent des photons absorbés, par rapport aux fluorophores conventionnels actuellement utilisés pour fabriquer des OLED dont les efficacités quantiques sont limitées à 25%. Cette forte augmentation de l'efficacité signifie que les fabricants pourraient produire des OLED à utiliser dans des appareils nécessitant une faible consommation d'énergie, tels que les téléphones portables, ce qui pourrait à son tour conduire à des développements futurs où pratiquement n'importe quelle surface peut être convertie en une source d'éclairage bon marché et économe en énergie couvrant de vastes zones de maisons, de bureaux et plus encore!

 Dans le présent travail, nous allons, utiliser la bibliothèque 

1. `rdkit` pour 
   1. examiner quelques propriétés des molécules d'interet, en mettant entre autre en exergue la propriété de faible donneur de ces molécules;
   2. obtenir la structure xyz optimisée (minimisation d'énergie moléculaire) en utilisant les champs de force moléculaires utilisés MMFF94 pour minimiser l'énergie potentielle de la molécule, en tenant compte des interactions intramoléculaires (liaisons, angles, torsions, interactions de van der Waals, etc.) et l'algorithme ETKDG (Experimental Torsion Knowledge Distance Geometry) pour la génération de conformères;
   3. évaluer l'alignement moléculaire;
2. `xTB` et `CREST` pour optimiser les structures moleculaires xyz en utilisant les méthodes semi-empiriques; 
3. `pyscf` pour effectuer les TDDFT (gap HOMO-LUMO, gap singulet-triplet, force d'oscillateurs) en prenant en compte la solvatation (pyscf.solvent.ddCOSMO) et la dispersion (dftd4.pyscf).

* Les matériaux utilisés dans les structures OLEDs ont la particularité d'être des semi-conducteurs organiques: ils peuvent conduire des porteurs de charges (électrons et/ou trous) et présentent une bande interdite (gap HOMO-LUMO) 
$$1.5{\rm eV}\leq E_g = E_{\rm LUMO} - E_{\rm HOMO}\leq 3.2{\rm eV},$$ 
ce qui est favorable à l'émission de lumière visible, respectivement du bleu (profond) au rouge.

* Pour nos molécules d'interêt, nous limitons cet état excité aux seuls états singulet $S_1$ et triplet $T_1$. Dans un état singulet, tous les électrons d'un système sont appariés en spin, ce qui ne leur donne qu'une seule orientation possible dans l'espace. Un état excité singulet ou triplet peut se former en excitant l'un des deux électrons à un niveau d'énergie supérieur. L'électron excité conserve la même orientation de spin dans un état excité singulet, alors que dans un état excité triplet, l'électron excité a la même orientation de spin que l'électron à l'état fondamental.

* Un ensemble de spins d'électrons n'est pas apparié dans un état triplet, ce qui signifie qu'il existe trois orientations possibles dans l'espace par rapport à l'axe. 

Une molécule OLED intéressante (voir Tartarus)
1. minimise le gap singulet-triplet $\Delta E(S_1-T_1)$; 
2. maximise la force d'oscillateur $f_{12}$ de la transition de fluorescence entre $S_1$ et $S_0$;
3. maximise la fonction multi-objective
$$+f_{12} - \Delta E(S_1-T_1)- |\Delta E(S_0-S_1) - 3.2{\rm eV}|.$$

In [19]:
import os
import time
import datetime
import subprocess as sp
from pathlib import Path
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Draw
from rdkit.Contrib.SA_Score import sascorer
from rdkit.Contrib.NP_Score import npscorer

from pyscf.data import nist
from pyscf import gto, scf, dft, tddft, sgx, solvent
import dftd4.pyscf as disp
from pyscf.lib import chkfile
from pyscf.tools import cubegen, molden

au2ev = nist.HARTREE2EV


### RDKit Calculations

In [20]:
def smiles_to_2d(smiles_dict: dict, working_dir: str):
    """Create RDKit 2D representation of molecule from SMILES as grid images
    and save them to a file, using RDKit

    Args:
        smiles_dict (dict): A dictionary containing SMILES strings as values
        working_dir (str): The directory path where images will be saved

    """

    # Create a list of RDKit Mol objects from the SMILES strings
    mol_list = [Chem.MolFromSmiles(smiles) for smiles in smiles_dict.values()]

    # Generate the 2D images of the molecules
    img_list = Draw.MolsToGridImage(
        mol_list,
        molsPerRow=2,
        legends=list(smiles_dict.keys()),
        subImgSize=(300, 200),
        returnPNG=False
    )

    # Save the 2D molecules images to a file
    path_img = working_dir / "Our_2D_Molecules.png"
    try:
        img_list.save(path_img)
    except Exception as e:
        print(f"Error occurred while saving the images: {e}")
        exit()


In [21]:
def mol_properties(smile_dict, working_dir):
    """Calculate molecule properties with RDKit and save to a CSV file

    Args:
        smiles_dict (dict): A dictionary containing SMILES strings as values
        working_dir (str): The directory path where results of calculations will be saved

    """

    properties_dict = {}

    for smi_key, value in smile_dict.items():
        mol = Chem.MolFromSmiles(value)
        properties = {}

        # Calculating all Descriptors
        properties['Molecular weight'] = Descriptors.ExactMolWt(mol)
        properties['Average molecular weight'] = Descriptors.HeavyAtomMolWt(mol)
        properties['Heavy atoms count'] = Descriptors.HeavyAtomCount(mol)
        properties['Solubility'] = Descriptors.MolLogP(mol)
        properties['Polarity'] = Descriptors.TPSA(mol)
        properties['Number of hdb'] = Descriptors.NumHDonors(mol)
        properties['Number of hab'] = Descriptors.NumHAcceptors(mol)
        fscore = npscorer.readNPModel()
        properties['NP_score'] = npscorer.scoreMol(mol,fscore)
        properties['Confidence value'] = npscorer.scoreMolWConfidence(mol,fscore)

        properties_dict[smi_key] = properties   

    # Create the properties dataframe
    df = pd.DataFrame.from_dict(properties_dict, orient='index')

    # Setting properties results path
    path_results = working_dir / f'Mol_Properties.csv'

    # Save the properties results dataframe to a file
    with open(path_results, "a") as f:
        f.write(f"\n\nThe molecules properties evaluated with RDKIT\n")
        df.to_csv(f)


### Structure 3D

Les optimisations de structure constituent la première étape de la plupart des investigations en chimie quantique, partant soit d'une référence expérimentale, soit d'une estimation de bas niveau pouvant utiliser la physique quantique semi-empirique ou des méthodes de champ de force. À ce stade, les erreurs d'utilisateur les plus typiques concernent la ou les structures de départ et les erreurs d'entrée entraînant des états de charge, de spin ou de protonation erronés, qui modifient radicalement les résultats, alors que les mauvais choix du modèle théorique sont moins courants, plus fréquents et moins sevère. Par conséquent, il est recommandé de vérifier soigneusement les entrées avant de mettre la machine computationnelle en mouvement. Les problèmes de convergence de champ auto-cohérents sont souvent une forte indication de problèmes avec l'entrée.

In [22]:
def generate_RDKit_3d_conformation(smi_key, smile, working_dir):
    """Generate RDKit 3D optimized conformation of a molecule from SMILES
        and store the xyz coordinates in a file

    Args:
        smi_key (str): The key associated with the molecule
        smile (str): The SMILES string of the molecule
        working_dir (str): The directory path where results of calculations will be saved

    Returns:
        mol_xyz (rdkit object): xyz coordinates of the RDKit molecule object
        with optimized 3D conformation

    """
   # Setting path for xyz file of mol_rdkit object
    path_xyz = working_dir / f'{smi_key}.xyz'

    if not smile:
        raise ValueError("Invalid SMILES string")

    # if not path_xyz.exists():
    # Defines a molecule from its SMILES string
    mol_rdkit = Chem.MolFromSmiles(smile)

    # Add explicit Hs
    mol_rdkit = Chem.AddHs(mol_rdkit)

    # Generates the initial 3D conformation of the molecule
    AllChem.EmbedMolecule(mol_rdkit)

    # Optimizes the 3D conformation of the molecule using MMFF - Merck Molecular Force Field
    AllChem.MMFFOptimizeMolecule(mol_rdkit, maxIters=200, mmffVariant="MMFF94s")

    #Canonicalize the orientation of the conformation
    Chem.rdMolTransforms.CanonicalizeMol(mol_rdkit, normalizeCovar=True, ignoreHs=False)

    # Convert RDKit molecule to XYZ format
    mol_xyz = Chem.MolToXYZBlock(mol_rdkit)

    # Writing optimized mol_xyz to a file
    try:
        with open(path_xyz, "w") as f:
            f.write(mol_xyz)
    except Exception as e:
        print(f"Error occurred while writing the XYZ file: {e}")
        exit()


### Tight-Binding optimization and calculation

Les options de xTB 
* "--gfn 2" requests the GFN2-xTB level of theory. This is actually the default, but here we specify it here for clarity.
* "--opt vtight" requests geometry optimization with vtight convergence criteria.
* "--parallel nproc --ignore=2" asks xtb to use only 'nproc --ignore=2' CPU (the remaining processors are for input/output).

Les options de CREST
* -T nproc --ignore=2 to also configure the number of CPU to use with CREST

Tip(CREST): It’s usually wise to pre-optimze your input structure with xtb at the same level on which the conformational search shall be conducted.

In [23]:
def clean_xtb_files():
    #---------------------------------------------------------------------
    # Clean up output files from xtb, xtb-stda and CREST processes
    #FIXME To call after each xtb function
    #---------------------------------------------------------------------
    sp.run(['rm', 'bondlengths', 'charges', 'coord', 'coord.original', 'cregen_0.tmp','spec.dat','rots.dat',
            'cregen_1.tmp', 'cre_members', 'crest_best.xyz', 'crest_conformers.xyz', 'vibspectrum', 'hessian', 'g98.out',
            'crest.energies', 'crest_rotamers.xyz', 'gfnff_charges', 'gfnff_topo', 'wfn.xtb', 'xtbhess.xyz',
            '.history.xyz', 'struc.xyz', 'wbo', 'xtbopt.xyz', 'xtbopt.log', '.xtboptok', 'crest_property.xyz',
            'xtbrestart', 'xtbtopo.mol', 'xtblast.xyz', 'gfnff_adjacency', '.UHF', 'tda.dat',
            'ensemble_energies.log', 'charges3', 'charges', 'molden.input', 'crest_0.mdrestart',
            'crest_dynamics.trj', 'crestopt.log', 'crest.restart', 'crest_input_copy.xyz'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
    # For folder
    sp.run(['rm', '-r', 'calculation.level.1'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

def clean_xtb_files2():
    #---------------------------------------------------------------------
    # Clean up output files from xtb, xtb-stda and CREST processes
    #FIXME To call after running crest function and leave crest_best.xyz file
    #---------------------------------------------------------------------
    sp.run(['rm', 'bondlengths', 'charges', 'coord', 'coord.original', 'cregen_0.tmp','spec.dat','rots.dat',
            'cregen_1.tmp', 'cre_members', 'crest_conformers.xyz', 'vibspectrum', 'hessian', 'g98.out',
            'crest.energies', 'crest_rotamers.xyz', 'gfnff_charges', 'gfnff_topo', 'wfn.xtb', 'xtbhess.xyz',
            '.history.xyz', 'struc.xyz', 'wbo', 'xtbopt.xyz', 'xtbopt.log', '.xtboptok', 'crest_property.xyz',
            'xtbrestart', 'xtbtopo.mol', 'xtblast.xyz', 'gfnff_adjacency', '.UHF', 'tda.dat',
            'ensemble_energies.log', 'charges3', 'charges', 'molden.input', 'crest_0.mdrestart',
            'crest_dynamics.trj', 'crestopt.log', 'crest.restart', 'crest_input_copy.xyz'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
    # For folder
    sp.run(['rm', '-r', 'calculation.level.1', 'PROP'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)


In [24]:
def xyz_opt_xtb(smi_key, smile, working_dir, phase='gas'):
        """
        Perform xtb optimization on the generated xyz coordinates.
        """
        # Setting path for xtb optimization files
        path_xtb_xyz = working_dir / f'{smi_key}_{phase}_xtb.xyz'
        path_xtb_opt_log = working_dir / f'{smi_key}_{phase}_xtb_opt.log'        
        path_xtb_Pre_opt_log = working_dir / f'{smi_key}_{phase}_xtb_Pre_opt.log'

        generate_RDKit_3d_conformation(smi_key, smile, working_dir)
        path_xyz = working_dir / f'{smi_key}.xyz'

        if phase == 'gas':
                # pre-optimization with xtb
                xtb_processP = sp.run(['xtb', path_xyz, '--gfn', '2', '--ohess', '--opt', 'vtight',
                                        '--parallel', 'nproc --ignore=2'],
                        stdout=sp.PIPE, stderr=sp.PIPE, text=True, universal_newlines=True)

                # Best conformation search with CREST
                sp.run(['crest', 'xtbopt.xyz', '--gfn2', '--mquick', '--prop', 'hess', '--noreftopo', 
                        '-T', 'nproc --ignore=2'],
                        stdout=sp.DEVNULL, stderr=sp.DEVNULL)
                clean_xtb_files2()

                # optimization with xtb
                xtb_process = sp.run(["xtb", "crest_best.xyz", "--gfn", "2", "--ohess", "--opt","vtight", 
                                      "--parallel", "nproc --ignore=2"],
                        stdout=sp.PIPE, stderr=sp.PIPE, text=True, universal_newlines=True)
        else:
                # pre-optimization with xtb
                xtb_processP = sp.run(['xtb', path_xyz, '--gfn', '2', '--ohess', '--opt', 'vtight', 
                                       '--parallel', 'nproc --ignore=2', '--alpb', phase],
                        stdout=sp.PIPE, stderr=sp.PIPE, text=True, universal_newlines=True)

                # Best conformation search with CREST
                sp.run(['crest', 'xtbopt.xyz', '--gfn2', '--mquick', '--prop', 'hess', '--noreftopo', 
                        '-T', "nproc --ignore=2", '--alpb', phase],
                        stdout=sp.DEVNULL, stderr=sp.DEVNULL)
                clean_xtb_files2()

                # optimization with xtb
                xtb_process = sp.run(["xtb", "crest_best.xyz", "--gfn", "2", "--ohess", "--opt","vtight", 
                                      "--parallel", "nproc --ignore=2", "--alpb", phase],
                        stdout=sp.PIPE, stderr=sp.PIPE, text=True, universal_newlines=True)

        # Store the xtbopt.xyz file in path_xtb_xyz
        sp.run(['cp', 'xtbopt.xyz', path_xtb_xyz], stdout=sp.DEVNULL, stderr=sp.DEVNULL)

        # Write the XYZ file that can be used in the pyscf object
        mol_xtb_xyz = path_xtb_xyz.read_text()
        lines = mol_xtb_xyz.strip().split('\n')
        mol_xtb_xyz = '\n'.join(lines[:1] + [''] + lines[2:])

        #view3D(mol_xtb_xyz, f'{smi_key} xTB Molecule', fmt='xyz')
                
        # Store xtb log file (pre-optimization)
        with open(path_xtb_Pre_opt_log, "w") as fl:
                fl.write(xtb_processP.stdout)

        # Store xtb log file (optimization)
        with open(path_xtb_opt_log, "w") as fl:
                fl.write(xtb_process.stdout)

        # Cleaning all tempory files
        clean_xtb_files()

In [25]:
def pyscf_molecule_object(smi_key, smile, working_dir, basis = 'def2-SVP', 
                          phase='gas', opt_xtb = True):
    """Create PySCF molecule object from mol_rdkit object

    Args:
        smi_key (str): The key associated with the molecule
        smile (str): The SMILES string of the molecule
        working_dir (str): The directory path where the XYZ file of
        molecule will be saved
        basis (str): The basis set used in PySCF. Default is 'def2-SVP'
        opt_xtb (bool): Whether to perform TB optimization

    Returns:
        mol_pyscf (pyscf.gto.Mole): The PySCF molecule object
    """

    # 3D optimized conformation
    if opt_xtb:
        xyz_opt_xtb(smi_key, smile, working_dir, phase=phase)
        path_xtb_xyz = working_dir / f'{smi_key}_{phase}_xtb.xyz'
        mol_xyz = path_xtb_xyz.read_text()
    else: 
        generate_RDKit_3d_conformation(smi_key, smile, working_dir)
        path_xyz = working_dir / f'{smi_key}_{phase}_xtb.xyz'
        mol_xyz = path_xyz.read_text()    
    
    # Remove the first 2 lines from XYZ data
    mol_xyz = '\n'.join(mol_xyz.strip().split('\n')[2:])

    # Convert mol_rdkit object to Pyscf Mol objects
    mol_pyscf = gto.Mole(
        atom=mol_xyz,
        charge=0,
        spin = 0,
        basis = basis,
        symmetry = True,
        unit = 'Angstrom')

    return mol_pyscf


### TDDFT-TDA calculations with solvation and dispersion


Dans les théories des champs auto-cohérentes (SCF), les énergies d'excitation des systèmes moléculaires peuvent être obtenu comme valeurs propres, $\omega_n$, du problème aux valeurs propres de la réponse :
$$
     \begin{pmatrix}
         \mathbf{A} & \mathbf{B} \\
         \mathbf{B}^\ast & \mathbf{A}^\ast
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X}_n \\ \mathbf{Y}_n
     \end{pmatrix} = \omega_n
     \begin{pmatrix}
         \mathbf{1} & \mathbf{0} \\
         \mathbf{0} & -\mathbf{1}
     \end{pmatrix}
     \begin{pmatrix}  \mathbf{X}_n \\ \mathbf{Y}_n
     \end{pmatrix},
$$    
où 
* $\mathbf{A}$ et $\mathbf{B}$ sont les orbitales les Hessiennes qui apparaissent également dans l'analyse de stabilité pour les états de référence,
\begin{equation}
\begin{split}
A_{aibj} &= \delta_{ij}f_{ab} - \delta_{ab}f_{ij} + (ai|jb) - \gamma (ab|ji) + w_{\mathrm{xc}; aijb} ,\\
B_{aibj} &= (ai|bj) - \gamma (aj|ib) + w_{\mathrm{xc}; aibj} ,
\end{split}
\end{equation}
avec $ij$ et $ab$ les orbitales occupées et virtuelles respectivement,
* $\omega_n$ est l'énergie d'excitation, et
* $\mathbf{X}_n$ et $\mathbf{Y}_n$ représentent la réponse de la matrice de densité ou l'opérateur statistique.

Dans les cas où le système possède un état fondamental dégénéré ou présente des instabilités triplet, les algorithmes utilisés pour résoudre le les équations ci-dessus peuvent être instables. Ceci peut être résolu en appliquant l'approximation de Tamm-Dancoff (TDA), où l'on néglige simplement les matrices $\mathbf{B}_n$ et $\mathbf{Y}_n$. Il en résulte un problème Hermitien aux valeurs propres
$$
\mathbf{AX}_n = \omega_n\mathbf{X}_n.
$$

Il est à noter que lorsque TDA est appliquée, dans la TDDFT, la réponse linéaire du potentiel de corrélation d'échange (XC) conduit à une dérivée d'ordre 2 de la fonctionnelle XC, qui n'apparaît pas dans la DFT de l'état fondamental.

Par défaut, seuls les états excités singulets sont calculés. Afin de calculer les états excités des triplets, il faut définir l'attribut *singlet* sur `False`.

Les différents modèle de solvation COSMO sont:  
- CPCM=Conductor-like polarizable continuum
- SMD= solvation model based on the molecular electron density (good)
- COSMO-RS=conductor-like screening model for real solvents 
- DCOSMO-RS= direct conductor-like screening model for real solvents (good)


### Solvatation

(Best Practices DFT)

Il de prendre en compte l'effet du mélange de la molécule avec un solvant. En effet, les molécules voisines d’un solide ou d’un soluté en solution peuvent avoir des effets drastiques sur la structure et les propriétés de l’ensemble du système. En conséquence, pour la chimie en phase condensée, un modèle de solvant approprié doit dans tous les cas être appliqué. 

L'approche la plus courante dans un contexte DFT consiste à utiliser des modèles de solvatation continue qui incluent implicitement l'interaction de la molécule avec le solvant via un potentiel effectif dans l'Hamiltonien. Cela signifie qu’aucune molécule de solvant réelle n’est présente dans le calcul. Les représentants courant de cette classe incluent :
* le modèle de continuum polarisable de type conducteur (CPCM, conductor-like polarizable continuum model),
* le modèle de solvatation basé sur la densité électronique moléculaire (SMD, solvation model based on the molecular electron density),
* le modèle de blindage de type conducteur (COSMO, conductor-like screening model for real solvents),
* le modèle de blindage de type conducteur pour les solvants réels (COSMO-RS, the conductor-like screening model for real solvents),
* le modèle de blindage direct de type conducteur pour les solvants réels (DCOSMO-RS, direct conductor-like screening model for real solvents).

Ces modèles diffèrent sous divers aspects. CPCM et COSMO sont des modèles purement électrostatiques, dépourvus de contributions provenant de la création de cavités qui coûtent de l'énergie dans le solvant, et d'interactions attrayantes de Van der Waals avec le solvant, qui conduisent à des erreurs substantielles si la surface accessible au solvant change de manière significative. SMD, COSMO-RS et DCOSMO-RS incluent de tels contributions et sont donc recommandées. **Néanmoins, il convient de noter que le modèle COSMO-RS ne peut pas être utilisé dans des optimisations géométriques ou des calculs de fréquence et doit être remplacé par DCOSMO-RS à cet effet**. D'autres méthodes de solvatation implicites remarquables qui peuvent également être utilisées avec des méthodes de physique quantique semi-empirique et de champ de force sont le modèle de Born généralisé avec surface accessible au solvant (GBSA, generalized-Born model with solvent-accessible surface area) et le modèle analytique linéarisé de Poisson–Boltzmann (ALPB, analytical linearized Poisson–Boltzmann model). 

Néanmoins, pour des cas spécifiques, l'inclusion de molécules de solvant explicites peut être nécessaire et les modèles de solvatation implicites deviennent insuffisants. Dans les approches de micro-solvatation, les molécules de solvant réelles sont placées aux positions importantes et les plus fortement liées d'un système. Les molécules de solvant explicites doivent être incluses lorsqu'elles sont fortement liées et/ou sont fortement impliquées dans le processus chimique/physique considéré. Ceci peut être testé à un niveau théorique inférieur par un exemple d'optimisation géométrique et d'évaluation énergétique pour un système modèle représentatif. Cependant, la solvatation explicite présente également ses inconvénients, car il peut être très difficile de faire converger les propriétés avec le nombre de molécules de solvant explicites. De plus, la surface d'énergie potentielle des systèmes explicitement solvatés est souvent plate et parsemée de minima locaux de différentes structures de solvants, ce qui rend les optimisations longues et fastidieuses. La résolution explicite doit donc être utilisée avec précaution. 

Négliger les effets de solvatation, en particulier pour les molécules polaires ou chargées, peut entraîner de grands écarts dans les calculs thermochimiques, voire même des structures électroniques fondamentalement erronées, par exemple pour les zwitterions.

#### Recommandations 
* Appliquer des modèles de solvatation implicites pour une molécule en solution ; il est préférable d'utiliser des modèles physiquement complets, tels que COSMO-RS ou SMD.
* Etre prudent avec les systèmes chargés pour lesquels les modèles continus peuvent être inexacts (plus la densité de charge est élevée, plus ils sont imprécis).
* Envisagez une résolution explicite si nécessaire.

In [26]:
def tddft_osc_strength_calculations(smi_key, smile, working_dir, basis = 'def2-SVP',
                                     phase = 'gas', opt_xtb = True, dispersion = True,
                                     verbose = 5):
    """PySCF TDDFT-TDA D4 calculations of fluorescence energy,
    singlet-triplet gap and oscillator strength with and without solvation

    Part of code is from https://github.com/aspuru-guzik-group/Tartarus

    Args:
        smi_key (str): smile key
        smile (str): The SMILES string of the molecule
        working_dir (str): The directory path where results of calculations will be saved
        phase (str) : 'gas' for the gas-phase or 'sol' for solvation

    """

    #=====================================
    # DFT CALCULATIONS
    #=====================================

   # Setting path for DFT results, chkfile and analyze object
    path_results = working_dir / f'Results_{smi_key}.csv'
    path_DFT_chk = working_dir / f'{smi_key}_{phase}_DFT.chk'
    path_DFT_log = working_dir / f'{smi_key}_{phase}_DFT.log'

    mol_pyscf = pyscf_molecule_object(smi_key, smile, working_dir, basis = basis, 
                          phase = phase, opt_xtb = opt_xtb)

    # To write output results of mdft.analysis on a file
    mol_pyscf.output = path_DFT_log

    # Singlets GS DFT calculations
    mdft = dft.RKS(mol_pyscf) # Create a PySCF DFT mean-field object
    # mdft.density_fit(auxbasis='def2-universal-jkfit') # To speed up calculations
    mdft = sgx.sgx_fit(mdft) # To speed up calculations
    # mdft.with_df.dfj = True # Using RI for Coulomb matrix while K-matrix is constructed with SGX method
    mdft._numint.libxc = dft.xcfun # Switch to xcfun because 3rd order GGA functional derivative is not available in libxc
    mdft.xc = 'B3LYP' #or R2SCAN, Exchange-Correlation functional
    mdft.nlc = 'VV10' # To specify the inclusion of the VV10 nonlocal correlation functional
    mdft.max_cycle = 512 # Maximum number of SCF iterations
    mdft.grids.level = 4  # Level of grid accuracy (higher is more accurate)
    mdft.conv_tol = 1E-7 # SCF convergence tolerance
    mdft.max_memory = 12000 # Maximum memory that can be allocated
    mdft.grids.becke_scheme=dft.gen_grid.original_becke # Use the Becke 'Good' grid
    mdft.init_guess = 'huckel' # Basically a tight-binding guess (see J. Chem. Theory Comput. 15, 1593 (2019))
    mdft.verbose = verbose

    if phase == 'sol':
    # Set up the COSMO solvation model ## CPCM, *SMD, COSMO-RS, *DCOSMO-RS
        mdft = mdft.run().ddCOSMO()
        mdft.with_solvent.eps = 2.3741   # Toluene

    if path_DFT_chk.exists():
        path_DFT_status = f'{path_DFT_chk} exist'
        start = time.time()

        # Load the DFT results from the checkpoint file
        mdft.__dict__.update(chkfile.load(path_DFT_chk, 'scf'))
        if dispersion:
            mdft = disp.energy(mdft).run()  
        else:
            mdft = mdft.run()   

        elapsed_DFT = str(datetime.timedelta(
            seconds = time.time() - start))
    else:
        path_DFT_status = f'{path_DFT_chk} don\'t exist'
        start = time.time()

        # Save the DFT results to a checkpoint file
        mdft.chkfile = path_DFT_chk
        if dispersion:
            mdft = disp.energy(mdft).run()  
        else:
            mdft = mdft.run()   

        if not mdft.converged:
            mdft = mdft.newton()
            if dispersion:
                mdft = disp.energy(mdft).run()  
            else:
                mdft = mdft.run()          

        elapsed_DFT = str(datetime.timedelta(
            seconds = time.time()- start))


    # Analyze the given DFT object
    mdft.analyze()

    # Index of HOMO and LUMO
    lumo_idx = mdft.mo_occ.tolist().index(0.)
    homo_idx = lumo_idx - 1 

    # Output cube files for active orbitals that can read by Vesta or XCryDen
    for i in [homo_idx, lumo_idx]:
        cubegen.orbital(
            mol_pyscf,
            str(working_dir / f'{smi_key}_{phase}_DFT_{i+1}.cube'),
            mdft.mo_coeff[:, i])
        
    # Output molden files for all orbitals that can read by Avogadro or Molden
    with open(working_dir / f'{smi_key}_{phase}_DFT.molden', 'w') as f:
        molden.header(mdft.mol, f)
        molden.orbital_coeff(mdft.mol, f, mo_coeff=mdft.mo_coeff, ene=mdft.mo_energy, occ=mdft.mo_occ)

    if disp:
        dd4 = disp.DFTD4Dispersion(mol_pyscf, xc='B3LYP').kernel()[0] # Compute the DFT-D4 dispersion correction
        dd4 = float(dd4)

    # Create the DFT results dataframe
    ##  One-electron Energy = 'e1'
    ##  Two-electron Coulomb Energy = 'coul'
    ##  DFT Exchange-Correlation Energy ='exc'
    ##  Nuclear Repulsion Energy ='nuc'
    list_results_DFT = [mdft.e_tot, # 'e1' + 'coul' + 'exc' + 'nuc'
                        (mdft.scf_summary['e1']+mdft.scf_summary['coul']+
                        mdft.scf_summary['exc']),#mdft.energy_elec()[0], # 'e1' + 'coul' + 'exc'
                        mdft.scf_summary['nuc'],
                        mdft.scf_summary['e1'],
                        mdft.scf_summary['coul'],
                        mdft.scf_summary['exc'],
                        dd4,
                        mdft.mo_energy[homo_idx], #mdft.mo_energy[homo_idx],
                        mdft.mo_energy[lumo_idx], #mdft.mo_energy[lumo_idx],
                        np.abs(mdft.mo_energy[homo_idx] - mdft.mo_energy[lumo_idx]),
                        homo_idx,
                        lumo_idx]
    dict_results_DFT = {'a.u.': list_results_DFT,
                    'eV': [x * au2ev for x in list_results_DFT]}
    df_DFT = pd.DataFrame(dict_results_DFT,
                        index = ['Total energy',
                                'Electronic energy',
                                'Nuclear repulsion energy',
                                'One-electron Energy',
                                'Two-electron Coulomb Energy',
                                'DFT Exchange-Correlation Energy',
                                'DFT-D4 dispersion correction',
                                'MO energy of HOMO',
                                'MO energy of LUMO',
                                'HOMO-LUMO gap',
                                'HOMO Index',
                                'LUMO Index'])

    #==================================
    # TDDFT-TDA CALCULATIONS
    #==================================

    # Singlets ES TDDFT calculations with Tamm-Dancoff-approximation
    #==================================
    mftda = tddft.TDA(mdft)
    mftda.nstates = 4
    if phase == 'sol':
        mftda = mftda.ddCOSMO()
        mftda.with_solvent.eps = 2.3741   # Toluene
        mftda.with_solvent.equilibrum_solvation = True

    # Setting path for singlets TDDFT-TDA chkfile
    path_TDA_chk = working_dir / f'{smi_key}_TDA.chk'

    if path_TDA_chk.exists():
        path_TDA_status = f'{path_TDA_chk} exist'
        start = time.time()

        # Load the TDA singlet results from the checkpoint file
        mftda.__dict__.update(chkfile.load(path_TDA_chk, 'tddft'))
        E_TDA = mftda.kernel()

        elapsed_TDA = str(datetime.timedelta(
            seconds = time.time() - start))
    else:
        path_TDA_status = f'{path_TDA_chk} don\'t exist'
        start = time.time()

        # Save the TDA singlet results to a checkpoint file
        mftda.chkfile = path_TDA_chk
        E_TDA = mftda.kernel()
        if not mftda.converged:
            mftda = tddft.TDA.newton()
            E_TDA = mftda.kernel()

        elapsed_TDA = str(datetime.timedelta(
            seconds = time.time()- start))

    # Singlet excitation energies
    ee_singlets = mftda.e

    # Singlet oscillators strength
    OS_singlets = mftda.oscillator_strength(gauge='length')


    # Triplets ES TDDFT calculations with Tamm-Dancoff-approximation
    #==================================
    mftda.singlet = False

    # Setting path for triplets TDDFT-TDA chkfile
    path_TDA3_chk = working_dir / f'{smi_key}_TDA3.chk'

    if path_TDA3_chk.exists():
        path_TDA3_status = f'{path_TDA3_chk} exist'
        start = time.time()

        # Load the TDA triplet results from the checkpoint file
        mftda.__dict__.update(chkfile.load(path_TDA3_chk, 'tddft'))
        E_TDA3 = mftda.kernel()

        elapsed_TDA3 = str(datetime.timedelta(
            seconds = time.time() - start))
    else:
        path_TDA3_status = f'{path_TDA3_chk} don\'t exist'
        start = time.time()

        # Save the TDA results to a checkpoint file
        mftda.chkfile = path_TDA3_chk
        E_TDA3 = mftda.kernel()

        elapsed_TDA3 = str(datetime.timedelta(
            seconds = time.time() - start))

    # Triplet excitation energies
    ee_triplets = mftda.e

    # Sought results
    #==================================
    #
    # fluorescence energy
    f_energy = min(ee_singlets)

    # Singlet-Tiplet gap
    gap_ST = min(ee_singlets) - min(ee_triplets)

    # Oscillator strength
    OStr = OS_singlets[0]

     # Multi-Objective function
    mobj = OStr - gap_ST * au2ev - np.abs(f_energy * au2ev - 3.2)

    #=====================================
    # DataFrame and saving in file
    #=====================================

    # Create the results dataframe
    list_results_TDA = [f_energy, gap_ST, OStr, mobj]
    dict_results_TDA = {'a.u.': list_results_TDA,
                'eV': [f_energy * au2ev,
                       gap_ST * au2ev,
                       OStr,
                       mobj]}

    df_TDA = pd.DataFrame(dict_results_TDA,
                        index = ['Fluorescence energy',
                                 'Singlet-Triplet gap',
                                 'Oscillator strength',
                                 'Multi-Obj'])

    # Save the DFT and TDDFT-TDA results dataframe to a file
    with open(path_results, "a") as f:
        f.write(f"\n\nDFT calculations of {smi_key} obtained in; \
            {elapsed_DFT}s ; as {path_DFT_status} \n")

        df_DFT.to_csv(f)

        f.write(f"\n\nTDDFT-TDA calculations of {smi_key} obtained in; \
                {elapsed_TDA}s ; as {path_TDA_status}\n \
                {elapsed_TDA3}s ; as {path_TDA3_status}\n")
        df_TDA.to_csv(f)



In [27]:
# def calculate_tadf_oled():
"""This function calculates various electronic structures concerning TADF OLED.
- ES0, ET1, ES1
- Fluorescence Energy and Singlet-Triplet energy gap
- Oscillator strength 
- with and without solvation and dispersion
Data files of each molecule are stored in a sub-directory.
"""

# Define a dictionary of SMILES strings and their corresponding names
smiles_dict = {
        'Test_AZB1': 'B1=CC=NC=C1'
        # 'DMAC-TRZ': 'CC1(C2=CC=CC=C2N(C3=CC=CC=C31)C4=CC=C(C=C4)C5=NC(=NC(=N5)C6=CC=CC=C6)C7=CC=CC=C7)C',
        # 'DMAC-DPS': 'CC1(C2=CC=CC=C2N(C3=CC=CC=C31)C4=CC=C(C=C4)S(=O)(=O)C5=CC=C(C=C5)N6C7=CC=CC=C7C(C8=CC=CC=C86)(C)C)C',
        # 'PSPCz': 'C1=CC=C(C=C1)S(=O)(=O)C2=CC=CC(=C2)N3C4=CC=CC=C4C5=CC=CC=C53',
        # '4CzIPN': 'C1=CC=C2C(=C1)C3=CC=CC=C3N2C4=C(C(=C(C(=C4C#N)N5C6=CC=CC=C6C7=CC=CC=C75)N8C9=CC=CC=C9C1=CC=CC=C18)N1C2=CC=CC=C2C2=CC=CC=C21)C#N',
        # 'Px2BP': 'C1=CC=C2C(=C1)N(C3=CC=CC=C3O2)C4=CC=C(C=C4)C(=O)C5=CC=C(C=C5)N6C7=CC=CC=C7OC8=CC=CC=C86',
        # 'CzS2': 'C1=CC=C2C(=C1)C3=CC=CC=C3N2C4=CC=C(C=C4)S(=O)(=O)C5=CC=C(C=C5)N6C7=CC=CC=C7C8=CC=CC=C86',
        # '2TCz-DPS': 'CC(C)(C)C1=CC2=C(C=C1)N(C3=C2C=C(C=C3)C(C)(C)C)C4=CC=C(C=C4)S(=O)(=O)C5=CC=C(C=C5)N6C7=C(C=C(C=C7)C(C)(C)C)C8=C6C=CC(=C8)C(C)(C)C',
        # 'TDBA-DI': 'B12C3=C(C=CC(=C3)C(C)(C)C)OC4=CC(=CC(=C41)OC5=C2C=C(C=C5)C(C)(C)C)N6C7=CC=CC=C7C8=C9C(=C1C(=C86)C2=CC=CC=C2N1C1=CC=CC=C1)C1=CC=CC=C1N9C1=CC=CC=C1'
}
# DMAC (dimethylacridine) donors
#
# 'DMAC-TRZ': CC1(C2=CC=CC=C2N(C3=CC=CC=C31)C4=CC=C(C=C4)C5=NC(=NC(=N5)C6=CC=CC=C6)C7=CC=CC=C7)C
# Compound CID: 118528399, MF: C36H28N4,
# IUPAC Name: 10-[4-(4,6-diphenyl-1,3,5-triazin-2-yl)phenyl]-9,9-dimethylacridine
# doi.org/10.1088/1361-6633/ace06
#
# 'DMAC-DPS': CC1(C2=CC=CC=C2N(C3=CC=CC=C31)C4=CC=C(C=C4)S(=O)(=O)C5=CC=C(C=C5)N6C7=CC=CC=C7C(C8=CC=CC=C86)(C)C)C,
# Compound CID: 59399558,  MF: C42H36N2O2S,
# IUPAC Name: 10-[4-[4-(9,9-dimethylacridin-10-yl)phenyl]sulfonylphenyl]-9,9-dimethylacridine
# doi.org/10.1088/1361-6633/ace06
#
# Cz (carbazole) donors
#
# PSPCz : C1=CC=C(C=C1)S(=O)(=O)C2=CC=CC(=C2)N3C4=CC=CC=C4C5=CC=CC=C53
# Compound CID: 141461768,  MF: C24H17NO2S,
# IUPAC Name: 9-[3-(benzenesulfonyl)phenyl]carbazole,  WIPO PubChem SID: 396604438
# Compound CID: 132916142, MF: C24H17NO2S (Good one)
# IUPAC Name: 9-[4-(benzenesulfonyl)phenyl]carbazole, (Good One)  doi.org/10.1038/s41524-021-00540-6
#
# '4CzIPN': C1=CC=C2C(=C1)C3=CC=CC=C3N2C4=C(C(=C(C(=C4C#N)N5C6=CC=CC=C6C7=CC=CC=C75)N8C9=CC=CC=C9C1=CC=CC=C18)N1C2=CC=CC=C2C2=CC=CC=C21)C#N,
# Compound CID: 102198498,  MF: C56H32N6,
# IUPAC Name: 2,4,5,6-tetra(carbazol-9-yl)benzene-1,3-dicarbonitrile, doi.org/10.1021/jacs.6b12124
#
# Benzophenone donor
#
# 'Px2BP': C1=CC=C2C(=C1)N(C3=CC=CC=C3O2)C4=CC=C(C=C4)C(=O)C5=CC=C(C=C5)N6C7=CC=CC=C7OC8=CC=CC=C86
# Compound CID: 1553685, MF: C37H24N2O3,
# IUPAC Name: bis(4-phenoxazin-10-ylphenyl)methanone
#
# carbazole/sulfone-based structures
#
# 'CzS2': C1=CC=C2C(=C1)C3=CC=CC=C3N2C4=CC=C(C=C4)S(=O)(=O)C5=CC=C(C=C5)N6C7=CC=CC=C7C8=CC=CC=C86
# Compound CID: 59711244, MF: C36H24N2O2S,
# IUPAC Name: 9-[4-(4-carbazol-9-ylphenyl)sulfonylphenyl]carbazole
#
# '2TCz-DPS': 'CC(C)(C)C1=CC2=C(C=C1)N(C3=C2C=C(C=C3)C(C)(C)C)C4=CC=C(C=C4)S(=O)(=O)C5=CC=C(C=C5)N6C7=C(C=C(C=C7)C(C)(C)C)C8=C6C=CC(=C8)C(C)(C)C'
# Compound CID: 90168099, MF: C52H56N2O2S
# IUPAC Name: 3,6-ditert-butyl-9-[4-[4-(3,6-ditert-butylcarbazol-9-yl)phenyl]sulfonylphenyl]carbazole
#
# Blue OLED with EQE of up to 38%
#  'TDBA-DI': 'B12C3=C(C=CC(=C3)C(C)(C)C)OC4=CC(=CC(=C41)OC5=C2C=C(C=C5)C(C)(C)C)N6C7=CC=CC=C7C8=C9C(=C1C(=C86)C2=CC=CC=C2N1C1=CC=CC=C1)C1=CC=CC=C1N9C1=CC=CC=C1'
# Compound CID: 137554973, MF: C62H48BN3O2
# IUPAC Name: 9-(4,18-ditert-butyl-8,14-dioxa-1-borapentacyclo[11.7.1.02,7.09,21.015,20]henicosa-2(7),3,5,9,11,13(21),15(20),16,18-nonaen-11-yl)-18,27-diphenyl-9,18,27-triazaheptacyclo[18.7.0.02,10.03,8.011,19.012,17.021,26]heptacosa-1,3,5,7,10,12,14,16,19,21,23,25-dodecaene, doi.org/10.1038/s41566-019-0415-5

# Verify if the "data" subdirectory exist and if not, create it
data_dir = Path('./data2406')
data_dir.mkdir(exist_ok=True)

# Create RDKit 2D representation of molecules from SMILES as grid images
# and save to a file
smiles_to_2d(smiles_dict, data_dir)

# Calculate molecules properties with RDKit and save to a CSV file
mol_properties(smiles_dict, data_dir)

for smi_key, smile in smiles_dict.items():

    #ph = ['gas', 'sol']
    ph = ['gas']
    for phase in ph:
        # Output directory
        working_dir = Path(f'./data2406/{phase}/{smi_key}')
        # Create the output directory if it doesn't exist
        working_dir.mkdir(parents=True, exist_ok=True)

        # Generate molecule xyz coordinates
        generate_RDKit_3d_conformation(smi_key, smile, working_dir)
        
        # # Generate PySCF molecule object
        # pyscf_molecule_object(smi_key, smile, working_dir, basis = 'def2-SVP', 
        #                   phase = phase, opt_xtb = True)
        
        # # Compute PySCF TDDFT-TDA fluorescence energy, singlet-triplet gap
        # # and oscillator strength
        tddft_osc_strength_calculations(smi_key, smile, working_dir, basis= 'def2-SVP',
                                     phase = phase, opt_xtb = True, dispersion=True)

reading NP model ...
model in
[17:47:59] UFFTYPER: Unrecognized atom type: B_1 (0)
[17:47:59] UFFTYPER: Unrecognized atom type: B_1 (0)


Initialize <pyscf.gto.mole.Mole object at 0x799195dc87f0> in <pyscf.dft.rks.RKS object at 0x799195dc8cd0>


overwrite output file: data2406/gas/Test_AZB1/Test_AZB1_gas_DFT.log
overwrite output file: data2406/gas/Test_AZB1/Test_AZB1_gas_DFT.log
overwrite output file: data2406/gas/Test_AZB1/Test_AZB1_gas_DFT.log


WARN: NLC functional found in DFT object.  Its second deriviative is not available. Its contribution is not included in the response function.
WARN: NLC functional found in DFT object.  Its second deriviative is not available. Its contribution is not included in the response function.


NameError: name 'df_DFT' is not defined